<a href="https://colab.research.google.com/github/tonyyoung3/data_analytics/blob/main/InvestmentStrategy_univariate_industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Investment Strategy

Date: Jan 25, 2022
Authors: af, jk, jv, gl

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os

# set directory to your data files
# path_data = 'data/'

# Please remember to upload the entire data folder to your Google Drive.
# If the data is uploaded to a different folder in your Google Drive, please change the "path_data" variable correspondingly!



In [ ]:
# os.listdir(path_data)

# If you see three data file names printed out below, namely 'CRSP_Monthly_2018.csv',
# 'Compustat_Annual_2018.csv', and 'F-F_Research_Data_Factors_2018.csv', then you have successfullly uploaded the data folder to Google Drive.



In [ ]:
#%%############################################################################
# Step 1: Preparing the CRSP file
###############################################################################
# print("Prepare CRSP file")
# t = time.time() # record the current time, so we can measure how long the code takes to run

# load data
crsp = pd.read_csv('data/crsp.csv')

# Have a look at the data
print(crsp.head())
print(crsp.dtypes)

   PERMNO      date TICKER     PRC        RET  SHROUT
0   10000  19851231    NaN     NaN        NaN     NaN
1   10000  19860131  OMFGA -4.3750          C  3680.0
2   10000  19860228  OMFGA -3.2500  -0.257143  3680.0
3   10000  19860331  OMFGA -4.4375   0.365385  3680.0
4   10000  19860430  OMFGA -4.0000  -0.098592  3793.0
PERMNO      int64
date        int64
TICKER     object
PRC       float64
RET        object
SHROUT    float64
dtype: object


In [ ]:
crsp.head()

,PERMNO,date,TICKER,PRC,RET,SHROUT
0,10000,19851231,NaN,NaN,NaN,NaN
1,10000,19860131,OMFGA,-4.3750,C,3680.0
2,10000,19860228,OMFGA,-3.2500,-0.257143,3680.0
3,10000,19860331,OMFGA,-4.4375,0.365385,3680.0
4,10000,19860430,OMFGA,-4.0000,-0.098592,3793.0


In [ ]:

### formatting ###
# make all variable names lowercase
crsp.columns = map(str.lower,crsp.columns)

# You should see that one of the important variables 'RET' (return) is not a number but 'object'.
# It is preferable to have this variable as a number, which Python denotes as float64 (float64 is just a special way of saying that a variable is a number)
# If you are interested search for 'floating point number'on internet. But it is computer-science issue!

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce')

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')

# Create separate 'year' and 'month' variables (we will use them later to merge CRSP with Compustat)
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap (unit of shrout is 1000)
crsp['mktcap'] = crsp['shrout'] * 1000 * crsp['prc'].abs()

In [ ]:
crsp.shape

(4818260, 9)

In [ ]:
crsp.head()

,permno,date,ticker,prc,ret,shrout,year,month,mktcap
0,10000,1985-12-31,NaN,NaN,NaN,NaN,1985,12,NaN
1,10000,1986-01-31,OMFGA,-4.3750,NaN,3680.0,1986,1,16100000.0
2,10000,1986-02-28,OMFGA,-3.2500,-0.257143,3680.0,1986,2,11960000.0
3,10000,1986-03-31,OMFGA,-4.4375,0.365385,3680.0,1986,3,16330000.0
4,10000,1986-04-30,OMFGA,-4.0000,-0.098592,3793.0,1986,4,15172000.0


In [ ]:
# check the missing fraction
print('Fraction of observations missing:')
print(1 - crsp.count() / len(crsp))

Fraction of observations missing:
permno    0.000000
date      0.000000
ticker    0.056784
prc       0.030939
ret       0.038870
shrout    0.008219
year      0.000000
month     0.000000
mktcap    0.030939
dtype: float64


In [ ]:
# Calculate the proportion of missing values for 'mktcap' per year
proportion_missing_mktcap_per_year = crsp.groupby('year')['mktcap'].apply(lambda x: x.isnull().mean())

# Order the years based on the proportion of missing 'mktcap' values
ordered_years_by_missing_mktcap_proportion = proportion_missing_mktcap_per_year.sort_values(ascending=False)

# Display the full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(ordered_years_by_missing_mktcap_proportion)

## decide to use data from 1980


year
1972    0.101908
1975    0.069150
1976    0.064677
1974    0.064188
1962    0.055251
1978    0.054595
1977    0.054372
1980    0.048241
1979    0.048008
1981    0.045130
1982    0.043743
1986    0.042385
1983    0.041992
2003    0.040555
2004    0.038929
1973    0.037827
2001    0.037471
2002    0.037285
2000    0.035846
2005    0.035606
1985    0.034948
1999    0.034666
1987    0.034182
1993    0.033156
2006    0.033146
1992    0.032929
1984    0.032713
2007    0.032691
1988    0.030223
1989    0.029567
1991    0.028993
1995    0.028543
1994    0.028214
1998    0.028190
2010    0.027971
2009    0.027792
1996    0.027553
1990    0.026844
1997    0.025772
2012    0.024964
2008    0.024886
1971    0.024719
2011    0.024712
2013    0.023739
1969    0.022708
1970    0.022213
2021    0.021968
2014    0.021113
1968    0.020341
2020    0.019363
2015    0.018585
2016    0.018202
2017    0.018090
2018    0.018028
2019    0.017059
1967    0.015369
2023    0.014684
1965    0.013980
1966    0

In [ ]:
#    For here, it is enough to simply drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])

# find the corresponding December data
crsp_december = crsp[crsp['month'] == 12]
crsp_december

crsp_december.shape


(406688, 9)

In [ ]:
crsp_december.head()

,permno,date,ticker,prc,ret,shrout,year,month,mktcap
0,10000,1985-12-31,NaN,NaN,NaN,NaN,1985,12,NaN
12,10000,1986-12-31,OMFGA,-0.51563,-0.377358,3843.0,1986,12,1981566.09
19,10001,1985-12-31,NaN,NaN,NaN,NaN,1985,12,NaN
31,10001,1986-12-31,GFGC,7.00000,0.015000,991.0,1986,12,6937000.00
43,10001,1987-12-31,GFGC,5.87500,-0.033535,992.0,1987,12,5828000.00


In [ ]:
# find the corresponding December data
#crsp_december = crsp[crsp['month'] == 12]
#crsp_december



### Some basic data cleaning ###
# keep only common shares
#crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
#crsp = crsp[crsp['exchcd'].isin([1,2,3])]

# make sure that there are no duplicates
# usually, we would investigate why there are duplicates and then decide which observation we want to keep
#    For here, it is enough to simply drop the duplicates.
#crsp = crsp.drop_duplicates(subset=['date','permno'])


#print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

In [ ]:
#create the december data for year end mktcap info

crsp_december = crsp_december[['permno','year','mktcap']]
crsp_december

crsp_december.to_csv('data/crsp_december.csv', index=False)

In [ ]:
#%%############################################################################
# Step 2: Preparing the Compustat (CCM) file
###############################################################################
#print("Prepare Compustat file")
#t = time.time() # reset our timer

ccm = pd.read_csv('data/ccm.csv')

In [ ]:
ccm.head()

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,dp,ib,txdb,costat
0,1000,25881,19701231,1970,INDL,C,D,STD,USD,1.352,1.878,0.000,I
1,1000,25881,19711231,1971,INDL,C,D,STD,USD,1.399,0.138,0.000,I
2,1000,25881,19721231,1972,INDL,C,D,STD,USD,1.200,1.554,0.288,I
3,1000,25881,19731231,1973,INDL,C,D,STD,USD,1.237,1.863,0.231,I
4,1000,25881,19741231,1974,INDL,C,D,STD,USD,1.326,1.555,0.091,I


In [ ]:
ccm.shape

(325290, 13)

In [ ]:
ccm_merge = pd.merge(ccm, crsp_december,how='left', left_on=['LPERMNO', 'fyear'], right_on=['permno', 'year'])
ccm_merge

# Have a look at the data
print(ccm_merge.head())
print(ccm_merge.dtypes)

ccm_merge.to_csv('ccm_merge.csv', index=False)


   GVKEY  LPERMNO  datadate  fyear indfmt consol popsrc datafmt curcd     dp  \
0   1000    25881  19701231   1970   INDL      C      D     STD   USD  1.352   
1   1000    25881  19711231   1971   INDL      C      D     STD   USD  1.399   
2   1000    25881  19721231   1972   INDL      C      D     STD   USD  1.200   
3   1000    25881  19731231   1973   INDL      C      D     STD   USD  1.237   
4   1000    25881  19741231   1974   INDL      C      D     STD   USD  1.326   

      ib   txdb costat   permno    year      mktcap  
0  1.878  0.000      I  25881.0  1970.0  26550000.0  
1  0.138  0.000      I  25881.0  1971.0  15266250.0  
2  1.554  0.288      I  25881.0  1972.0  13606875.0  
3  1.863  0.231      I  25881.0  1973.0   4646250.0  
4  1.555  0.091      I  25881.0  1974.0   4711125.0  
GVKEY         int64
LPERMNO       int64
datadate      int64
fyear         int64
indfmt       object
consol       object
popsrc       object
datafmt      object
curcd        object
dp          flo

In [ ]:
ccm_merge.shape

(325290, 16)

In [ ]:
### formatting ###
# make all variable names lowercase
ccm_merge.columns = map(str.lower,ccm_merge.columns)

# Change the dateformat
ccm_merge['datadate'] = pd.to_datetime(ccm_merge['datadate'], format='%Y%m%d')

# Create separate 'year' and 'month' variables
ccm_merge['year'] = ccm_merge['datadate'].apply(lambda x: x.year)
ccm_merge['month'] = ccm_merge['datadate'].apply(lambda x: x.month)


### Some basic data cleaning ###
# make sure that there are no duplicates (same as above)
ccm_merge = ccm_merge.drop_duplicates(subset=['datadate','gvkey'])
ccm_merge = ccm_merge.drop_duplicates(subset=['year','gvkey'])
ccm_merge = ccm_merge.drop_duplicates(subset=['year','lpermno'])

### Calculate the variables we will use for sorting ###
# Create lagged asset variable
# Note 1) Pandas does not know the panel data structure, so we need to make sure that the previous
#    record belongs to the same gvkey, and that there are no gaps in the data
# Note 2) We can use the backslash "\" do break long lines
ccm_merge = ccm_merge.sort_values(['gvkey','datadate']) # sort data by gvkey and date


ccm_merge['cf'] = (ccm_merge['ib']+ ccm_merge['dp'].fillna(0)+ccm_merge['txdb'].fillna(0))*1000000

#use cf and mkt data of last year to build porfolios
ccm_merge['cf_lagged'] = ccm_merge['cf'].shift(1)
ccm_merge['mktcap_lagged'] = ccm_merge['mktcap'].shift(1)

# only use the previous record if it 1) belongs to the same gvkey and 2) is one year older
ccm_merge.loc[(ccm_merge['gvkey'].shift(1) != ccm_merge['gvkey']) | \
        (ccm_merge['year'].shift(1) != ccm_merge['year']-1) | \
        (ccm_merge['month'].shift(1) != ccm_merge['month']),['cf_lagged','mktcap_lagged']] = np.NAN

ccm_merge['cfp'] = ccm_merge['cf_lagged']/ccm_merge['mktcap_lagged']

ccm_merge.head()



,gvkey,lpermno,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,dp,ib,txdb,costat,permno,year,mktcap,month,cf,cf_lagged,mktcap_lagged,cfp
0,1000,25881,1970-12-31,1970,INDL,C,D,STD,USD,1.352,1.878,0.000,I,25881.0,1970,26550000.0,12,3230000.0,NaN,NaN,NaN
1,1000,25881,1971-12-31,1971,INDL,C,D,STD,USD,1.399,0.138,0.000,I,25881.0,1971,15266250.0,12,1537000.0,3230000.0,26550000.0,0.121657
2,1000,25881,1972-12-31,1972,INDL,C,D,STD,USD,1.200,1.554,0.288,I,25881.0,1972,13606875.0,12,3042000.0,1537000.0,15266250.0,0.100680
3,1000,25881,1973-12-31,1973,INDL,C,D,STD,USD,1.237,1.863,0.231,I,25881.0,1973,4646250.0,12,3331000.0,3042000.0,13606875.0,0.223563
4,1000,25881,1974-12-31,1974,INDL,C,D,STD,USD,1.326,1.555,0.091,I,25881.0,1974,4711125.0,12,2972000.0,3331000.0,4646250.0,0.716922


In [ ]:
ccm_merge.shape

(321898, 21)

In [ ]:
# It is useful to know how many observations are missing
print('Fraction of observations missing:')
print(1 - ccm_merge.count() / len(ccm_merge))


#print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block


Fraction of observations missing:
gvkey            0.000000
lpermno          0.000000
datadate         0.000000
fyear            0.000000
indfmt           0.000000
consol           0.000000
popsrc           0.000000
datafmt          0.000000
curcd            0.000000
dp               0.110895
ib               0.068761
txdb             0.143462
costat           0.000000
permno           0.006359
year             0.000000
mktcap           0.017683
month            0.000000
cf               0.068761
cf_lagged        0.156025
mktcap_lagged    0.105363
cfp              0.168553
dtype: float64


In [ ]:
# Calculate the proportion of missing values for 'mktcap' per year
proportion_missing_cfp_per_year = ccm_merge.groupby('year')['cfp'].apply(lambda x: x.isnull().mean())

# Order the years based on the proportion of missing 'mktcap' values
ordered_years_by_missing_cfp_ratio = proportion_missing_cfp_per_year.sort_values(ascending=False)

# Display the full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(ordered_years_by_missing_cfp_ratio)

## decide to use data from 1980

year
1950    1.000000
1962    0.365801
1972    0.297579
1993    0.277831
1951    0.240000
1992    0.238283
1973    0.232129
1987    0.231662
2021    0.228938
1991    0.208812
1986    0.208672
1994    0.206329
1996    0.204971
2024    0.204545
1988    0.201186
1990    0.197559
2014    0.192745
1989    0.185458
1995    0.184669
2007    0.183996
2013    0.183716
2020    0.182630
1966    0.181935
1984    0.176678
1999    0.174975
1997    0.173675
2000    0.173181
1983    0.170075
2006    0.169857
1974    0.169187
2004    0.168939
2005    0.168802
2018    0.167496
2011    0.166604
2010    0.165837
2017    0.164755
2015    0.163454
2012    0.160313
2019    0.159370
1985    0.159311
1998    0.158479
2016    0.152046
2008    0.149390
1963    0.147974
1981    0.147052
2009    0.146496
1982    0.142112
1960    0.139655
2003    0.138093
2022    0.129861
2002    0.124963
2001    0.123415
1979    0.120747
1965    0.120214
1969    0.112525
1964    0.111966
2023    0.110989
1980    0.109125
1968    0

In [ ]:
ccm.head()

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,dp,ib,txdb,costat
0,1000,25881,19701231,1970,INDL,C,D,STD,USD,1.352,1.878,0.000,I
1,1000,25881,19711231,1971,INDL,C,D,STD,USD,1.399,0.138,0.000,I
2,1000,25881,19721231,1972,INDL,C,D,STD,USD,1.200,1.554,0.288,I
3,1000,25881,19731231,1973,INDL,C,D,STD,USD,1.237,1.863,0.231,I
4,1000,25881,19741231,1974,INDL,C,D,STD,USD,1.326,1.555,0.091,I


In [ ]:
#%%############################################################################
# Step 3: Preparing the industry file
###############################################################################
#print("Prepare Compustat file")
#t = time.time() # reset our timer

ind_sec = pd.read_csv('data/sec_industry.csv')
ind_sec.head()

/var/folders/_b/v51rk4qj7hxcq50ftvsf39540000gn/T/ipykernel_1571/485701793.py:7: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_sec = pd.read_csv('data/sec_industry.csv')


,PERMNO,date,SICCD,COMNAM,HSICCD,HSICMG,HSICIG
0,10000,19851231,NaN,NaN,3990,NaN,NaN
1,10000,19860131,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN
2,10000,19860228,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN
3,10000,19860331,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN
4,10000,19860430,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN


In [ ]:
ind_sec['year'] = ind_sec['date'].apply(lambda x: str(x)[:4])
ind_sec['month'] = ind_sec['date'].apply(lambda x: str(x)[4:6])
ind_sec['year'] = ind_sec['year'].astype(int)
ind_sec['month'] = ind_sec['month'].astype(int)
ind_sec = ind_sec[ind_sec['month'] == 12]
ind_sec = ind_sec.sort_values(by=['PERMNO', 'year'])

ind_sec['HSICCD_lagged'] = ind_sec['HSICCD'].shift(1)

# only use the previous record if it 1) belongs to the same gvkey and 2) is one year older
ind_sec.loc[(ind_sec['PERMNO'].shift(1) != ind_sec['PERMNO']) | \
        (ind_sec['year'].shift(1) != ind_sec['year']-1) | \
        (ind_sec['month'].shift(1) != ind_sec['month']),'HSICCD_lagged'] = np.NAN

ind_sec.head()

,PERMNO,date,SICCD,COMNAM,HSICCD,HSICMG,HSICIG,year,month,HSICCD_lagged
0,10000,19851231,NaN,NaN,3990,NaN,NaN,1985,12,NaN
12,10000,19861231,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN,1986,12,3990
19,10001,19851231,NaN,NaN,4925,NaN,NaN,1985,12,NaN
31,10001,19861231,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1986,12,4925
43,10001,19871231,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1987,12,4925


In [ ]:
ind_sec = ind_sec[ind_sec['HSICCD_lagged'].apply(lambda x: len(str(x)) == 4)]
ind_sec = ind_sec.dropna(subset=['HSICCD_lagged'])

ind_sec['HSICCD_lagged_two_digits'] = ind_sec['HSICCD_lagged'].astype(int) // 100
ind_sec.head()

,PERMNO,date,SICCD,COMNAM,HSICCD,HSICMG,HSICIG,year,month,HSICCD_lagged,HSICCD_lagged_two_digits
12,10000,19861231,3990.0,OPTIMUM MANUFACTURING INC,3990,NaN,NaN,1986,12,3990,39
31,10001,19861231,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1986,12,4925,49
43,10001,19871231,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1987,12,4925,49
55,10001,19881230,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1988,12,4925,49
67,10001,19891229,4920.0,GREAT FALLS GAS CO,4925,NaN,NaN,1989,12,4925,49


In [ ]:
#%%############################################################################
# Step 3: Sort stocks into portfolios and calculate returns
###############################################################################
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1981,2017).
#    You can wrap any list by the tqdm command to display a progress bar while looping over the list
portfolios = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1995,2023),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(crsp[crsp['year']==year-1]['permno'].unique())

    #can comment out below two codes if not running for industry
    #==========================================================================
    ind_filter = list(ind_sec[(ind_sec['HSICCD_lagged_two_digits'] == 73) & (ind_sec['year'] == year-1)]['PERMNO'].unique())
    permno_list = list(set(permno_list) & set(ind_filter))
    #==========================================================================

    # get the sorting variable for these companies at t-1
    sorting_data = ccm_merge.loc[(ccm_merge['year']==(year-1)) & \
                           (ccm_merge['lpermno'].isin(permno_list)), \
                           ['gvkey','lpermno','cfp']]

    # sort into 5 baskets by cashflow over assets
    nportfolios = 10 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['cfp'],nportfolios, labels=False)

    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]

    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = []
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'lpermno'].tolist()

        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]

        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row

        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)

        # collect portfolio returns in dec_port
        portfolios_window += [return_port]

    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)

    # collect results in portfolios
    portfolios += [portfolios_window]

# merge the returns from all windows
portfolios = pd.concat(portfolios,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years: 100%|██████████| 28/28 [00:01<00:00, 22.86it/s]

Step 3 completed in 1.2s


In [ ]:
portfolios
portfolios.to_csv('portfolio_returns.csv')


In [ ]:
portfolios

,0,1,2,3,4,5,6,7,8,9
date,,,,,,,,,,
1995-07-31,0.107407,0.065407,0.091028,0.093529,0.085539,0.090085,0.027441,0.048750,0.061638,0.048417
1995-08-31,0.169952,0.018065,0.010237,0.021052,0.023856,0.017696,0.059217,0.057513,0.047200,0.060889
1995-09-29,0.044278,0.111674,0.111494,0.055833,0.104178,0.041090,0.083240,0.047017,0.047141,0.048692
1995-10-31,-0.028166,-0.093252,-0.097248,-0.067975,-0.076310,-0.053750,-0.026616,-0.020680,-0.023153,-0.060254
1995-11-30,0.068564,0.037407,0.003653,-0.001466,0.018107,0.016760,0.020521,0.120917,0.035818,0.020120
1995-12-29,0.061944,0.014059,-0.006456,-0.012955,-0.029328,0.009986,0.070977,0.018555,0.020060,0.001811
1996-01-31,-0.021399,0.049014,0.030337,0.012024,0.012140,0.024904,0.072545,-0.035857,-0.001914,0.052199
1996-02-29,0.135314,0.011070,0.125171,0.107603,0.052356,0.052600,0.101865,0.020155,0.043940,0.045528
1996-03-29,-0.020501,-0.060078,0.042704,0.003656,-0.021762,0.088069,0.047786,0.043524,0.053073,0.051193


In [ ]:
print(ff.dtypes)


ExMkt    float64
SMB      float64
HML      float64
RF       float64
year       int64
month      int64
dtype: object


In [ ]:
#%%############################################################################
# Step 4: Performance Evaluation
# Step 4a: Merge Portfolio returns with Fama French data
###############################################################################

### load and prepare fama french data ###
# load Fama French monthly factors
ff = pd.read_csv('data/F-F_Research_Data_Factors_2024.csv')

# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'DATE':'date'},axis=1,inplace=True)

# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100


### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff = portfolios.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff['year'] = portfolios_ff.index.year
portfolios_ff['month'] = portfolios_ff.index.month

# merge
portfolios_ff = pd.merge(portfolios_ff,ff,on=['year','month'])



In [ ]:
portfolios_ff.rename(columns={col: str(i) for i, col in enumerate(portfolios_ff.columns[:100])}, inplace=True)


In [ ]:
portfolios_ff

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,year,month,ExMkt,SMB,HML,RF
0,-0.006451,0.034073,-0.034351,0.115122,-0.076192,-0.017872,-0.026408,-0.000050,0.011715,-0.006099,-0.012952,-0.043259,0.023378,-0.014425,-0.032774,0.027436,-0.009059,-0.035359,-0.050262,-0.033894,-0.014949,-0.039136,-0.003022,0.029139,-0.058044,-0.037644,-0.015174,-0.011630,-0.017024,-0.043294,0.059818,-0.064606,-0.033619,0.014625,-0.028562,-0.062368,-0.076768,-0.041504,-0.019659,-0.055538,-0.078839,-0.050793,-0.036183,-0.068531,-0.041591,-0.071675,-0.031083,-0.042034,-0.060312,-0.008889,-0.041181,-0.055653,-0.057662,-0.040939,-0.033582,-0.036172,-0.007785,-0.041065,-0.049051,-0.026656,-0.026559,-0.051508,-0.038782,-0.054279,-0.040897,-0.044086,-0.049805,-0.055545,-0.041747,0.010538,-0.008583,-0.047341,-0.004637,-0.036857,-0.051402,-0.054723,-0.035636,-0.007964,-0.020351,-0.037969,0.032693,-0.131322,-0.029211,-0.039721,-0.057840,-0.045369,-0.022235,-0.042478,-0.027922,-0.019805,0.007082,-0.068657,-0.044976,-0.018318,-0.019192,-0.032966,-0.027877,-0.029311,0.002669,0.009146,1990,7,-0.0190,-0.0312,-0.0003,0.0068
1,-0.098079,-0.026047,-0.078590,-0.125157,-0.085385,0.010837,-0.118388,-0.017559,-0.046558,-0.038968,-0.143055,-0.052851,-0.091884,-0.114524,-0.090367,-0.077295,-0.042918,-0.115071,-0.076499,-0.051705,-0.124225,-0.106070,-0.116225,-0.142017,-0.166366,-0.128895,-0.073486,-0.123746,-0.060851,-0.163240,-0.146219,-0.104285,-0.137011,-0.080090,-0.092219,-0.091619,-0.137227,-0.105415,-0.110674,-0.091558,-0.219368,-0.161048,-0.173899,-0.176123,-0.141211,-0.114021,-0.132658,-0.136023,-0.070597,-0.087911,-0.145159,-0.151529,-0.136728,-0.111751,-0.142634,-0.143169,-0.163319,-0.145345,-0.116471,-0.098821,-0.157126,-0.187707,-0.153945,-0.129385,-0.121117,-0.120727,-0.142710,-0.117165,-0.095003,-0.097747,-0.163788,-0.101880,-0.118089,-0.125367,-0.119402,-0.128494,-0.132340,-0.128388,-0.101810,-0.091628,0.031582,-0.195610,-0.095661,-0.116722,-0.098346,-0.116103,-0.140394,-0.125619,-0.122125,-0.081085,-0.204520,-0.197209,-0.094924,-0.089570,-0.100781,-0.109144,-0.101477,-0.113776,-0.091659,-0.079461,1990,8,-0.1015,-0.0357,0.0164,0.0066
2,-0.005103,-0.009985,-0.037983,0.017726,0.033871,-0.057310,0.022811,-0.075182,-0.057680,-0.093523,-0.090646,-0.054255,-0.131144,-0.074767,-0.048802,-0.056428,-0.044622,0.022288,-0.031482,-0.016369,-0.092632,-0.064718,-0.098995,-0.109933,-0.087293,-0.081032,-0.087702,-0.083386,-0.097740,-0.108664,-0.082810,-0.085994,-0.088615,-0.102106,-0.111946,-0.073122,-0.086944,-0.104292,-0.037384,-0.082772,-0.099875,-0.073662,-0.068787,-0.074776,-0.070549,-0.119588,-0.104062,-0.069538,-0.081966,-0.076171,-0.110808,-0.164791,-0.106823,-0.102684,-0.091664,-0.084623,-0.064860,-0.099498,-0.073664,-0.099492,-0.198830,-0.152845,-0.097470,-0.111991,-0.096098,-0.105690,-0.113522,-0.083338,-0.099554,-0.109136,-0.130604,-0.112073,-0.088554,-0.079654,-0.105114,-0.074512,-0.101471,-0.084003,-0.067489,-0.062482,-0.137425,-0.131424,-0.143289,-0.095518,-0.098796,-0.081951,-0.104790,-0.093763,-0.064496,-0.049165,-0.132857,-0.092385,-0.075887,-0.069522,-0.080643,-0.100017,-0.063217,-0.099595,-0.058551,-0.035354,1990,9,-0.0612,-0.0365,0.0064,0.0060
3,-0.060599,-0.067807,-0.086724,-0.008721,-0.032295,-0.032559,-0.082559,-0.042360,0.023209,-0.033058,-0.054960,-0.049508,-0.059421,-0.033457,-0.063013,-0.035868,-0.066329,-0.116740,-0.133225,-0.058869,-0.070384,-0.130261,-0.043018,-0.039135,-0.079120,0.022732,-0.068754,-0.046443,-0.042628,0.025196,-0.112326,-0.023537,-0.022176,-0.065853,-0.040122,-0.092527,-0.069750,-0.045349,-0.024155,-0.075466,-0.081674,-0.029801,-0.098339,-0.060197,-0.027058,-0.058621,-0.064642,-0.094577,-0.048881,-0.010209,-0.108555,-0.112734,-0.062453,-0.065224,-0.048902,-0.100455,-0.038145,-0.054623,-0.

In [ ]:
#%%############################################################################
# Step 4b: Regressions
###############################################################################

# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios.mean(axis=0))**12-1)*100)



# Calculate the excess returns
for p in range(100):
    portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']


### Market model regressions ###
table_capm = []
for p in range(nportfolios):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])

    table_capm += [table_row]

# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'

# show results
print("CAPM\n",table_capm)


### Three Factor model regressions ###
table_ff = []
for p in range(100):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])

    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'quintile'

# show results
print("Fama-French 3\n",table_ff)


Average returns (annualized percent)
 mktrk0cfork0    24.692285
mktrk0cfork1    25.121579
mktrk0cfork2    35.871781
mktrk0cfork3    24.173922
mktrk0cfork4    32.054147
mktrk0cfork5    34.447121
mktrk0cfork6    24.876587
mktrk0cfork7    26.514550
mktrk0cfork8    26.905359
mktrk0cfork9    32.842462
mktrk1cfork0    17.463636
mktrk1cfork1    16.663124
mktrk1cfork2    17.131230
mktrk1cfork3    16.841940
mktrk1cfork4    16.819366
mktrk1cfork5    20.425457
mktrk1cfork6    21.281112
mktrk1cfork7    18.383270
mktrk1cfork8    23.700208
mktrk1cfork9    19.092370
mktrk2cfork0    14.352877
mktrk2cfork1    14.178684
mktrk2cfork2    12.442643
mktrk2cfork3    13.571705
mktrk2cfork4    15.010844
mktrk2cfork5    15.624361
mktrk2cfork6    16.419947
mktrk2cfork7    16.603749
mktrk2cfork8    15.918342
mktrk2cfork9    15.083868
mktrk3cfork0    17.733066
mktrk3cfork1    12.023649
mktrk3cfork2    13.052256
mktrk3cfork3    17.137968
mktrk3cfork4    14.894459
mktrk3cfork5    18.804669
mktrk3cfork6    16.240654


/var/folders/_b/v51rk4qj7hxcq50ftvsf39540000gn/T/ipykernel_1894/848239117.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']
/var/folders/_b/v51rk4qj7hxcq50ftvsf39540000gn/T/ipykernel_1894/848239117.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']
/var/folders/_b/v51rk4qj7hxcq50ftvsf39540000gn/T/ipykernel_1894/848239117.py:12: PerformanceWarning: DataFrame is highly fragmente

In [ ]:
port_name = portfolios.mean(axis=0).index

table_ff['port_name'] = port_name
table_ff['market_group'] = table_ff['port_name'].str[5]
table_ff['cf_group'] = table_ff['port_name'].str[-1]
table_ff

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,rmse,R2,port_name,market_group,cf_group
quintile,,,,,,,,,,
0,0.007810,0.995947,1.499939,0.109661,1.931357,0.079303,0.450834,mktrk0cfork0,0,0
1,0.010971,0.677403,1.172259,-0.088863,2.877418,0.074774,0.342853,mktrk0cfork1,0,1
2,0.017290,0.747402,1.198274,0.037070,4.679380,0.072466,0.374391,mktrk0cfork2,0,2
3,0.009260,0.806088,0.925733,0.187503,2.665796,0.068124,0.356676,mktrk0cfork3,0,3
4,0.015505,0.662870,1.000101,0.140378,4.533044,0.067081,0.334490,mktrk0cfork4,0,4
5,0.016863,0.636562,1.170492,0.276028,4.229551,0.078189,0.296402,mktrk0cfork5,0,5
6,0.011154,0.615965,0.822684,0.213169,3.941224,0.055503,0.356763,mktrk0cfork6,0,6
7,0.011943,0.621506,1.016836,0.274222,4.329237,0.054101,0.422519,mktrk0cfork7,0,7
8,0.011179,0.753778,0.954106,0.378076,3.494783,0.062733,0.382242,mktrk0cfork8,0,8


In [ ]:
rtn = ((1+portfolios.mean(axis=0))**12-1)*100
type(rtn)

pandas.core.series.Series

In [ ]:
rtn['market_group'] = rtn['index'].str[5]
rtn['cf_group'] = rtn['index'].str[-1]

rtn.rename(columns={0: 'return'}, inplace=True)
rtn


,index,return,market_group,cf_group
0,mktrk0cfork0,22.747599,0,0
1,mktrk0cfork1,37.161881,0,1
2,mktrk0cfork2,32.107618,0,2
3,mktrk0cfork3,28.887009,0,3
4,mktrk0cfork4,27.162844,0,4
...,...,...,...,...
95,mktrk9cfork5,12.606787,9,5
96,mktrk9cfork6,12.808275,9,6
97,mktrk9cfork7,12.108197,9,7
98,mktrk9cfork8,12.555602,9,8
